In [5]:
#Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import ipywidgets as widgets
from IPython.display import display
from flask import Flask, request, jsonify

In [2]:
#Conversions
conversions = {
    "kWh_MWh": 1/10**3,
    "kWh_GWh": 1/10**6,
    "kWh_TWh": 1/10**9,
    "Mwh_TWh": 1/10**6,
    "g_kt": 1/10**9,
    "kg_t": 1/10**3,
    "t_kt": 1/10**3,
    "t_Mt": 1/10**6,
    "Mt_kt": 10**3,
    "km_Tkm": 1/10**12,
    "Mv_v": 10**6
}

In [3]:
#Total Metals
df_Metals = pd.read_excel('Metals.xlsx',sheet_name='Metals')
df_Power_Plants = pd.read_excel('Metals.xlsx',sheet_name='Power Systems')
df_Vehicles = pd.read_excel('Metals.xlsx',sheet_name='Vehicles')

In [6]:
app = Flask(__name__)

@app.route('/update_world_inputs', methods=['POST'])
def update_world_inputs():
    # Parse variables from request
    data = request.json
    World_GHG_Emissions = data.get('World_GHG_Emissions', 49758)  # default value is the original constant
    World_Power_Generation = data.get('World_Power_Generation', 28466)  # default value is the original constant
    Power_to_Remove_GHG = data.get('Power_to_Remove_GHG', 0.51)  # default value is the original constant
    Current_Year = data.get('Current_Year', 2023)  # default value is the original constant
    Years_To_Develop = data.get('Years_To_Develop', 10)  # default value is the original constant
    Year_To_Net_Zero = data.get('Year_To_Net_Zero', 2050)  # default value is the original constant

    # Recalculate dependent variables
    Years_To_Mine = Year_To_Net_Zero-Current_Year-Years_To_Develop
    GHG_data = data.get('GHG_data', {'Sector': ['Oil','Gas','Coal','Biomass','Industry','Waste','Agriculture, Forestry and Land Use', 'Other'],
                                     'Percentage': [23.2, 16.3, 33.1, 0.6, 5.2, 3.2, 18.4, 100 - 23.2 - 16.3 - 33.1 - 0.6 - 5.2 - 3.2 - 18.4]})
    GHG_df = pd.DataFrame(GHG_data)
    GHG_df['Absolute'] = GHG_df['Percentage']/100 * World_GHG_Emissions

    Power_data = data.get('Power_data', {'Source': ['Oil','Gas','Coal','Nuclear','Hydro','Renewables','Other'],
                                         'Percentage': [2.5, 22.9, 36.0, 9.8, 15.0, 12.8, 100 - 2.5 - 22.9 - 36.0 - 9.8 - 15.0 - 12.8]})
    Power_df = pd.DataFrame(Power_data)
    Power_df['Absolute'] = Power_df['Percentage']/100 * World_Power_Generation
    GHG_Power_TWh = Power_df['Absolute'].loc[Power_df['Source'].isin(['Oil','Gas','Coal'])].sum()

    # Add in energy costs for industry and waste
    Industry_Power = GHG_df.loc[GHG_df['Sector'] == 'Industry', 'Absolute'].values[0]*Power_to_Remove_GHG
    Waste_Power = GHG_df.loc[GHG_df['Sector'] == 'Waste', 'Absolute'].values[0]*Power_to_Remove_GHG

    power_requirements = pd.DataFrame({'Non-Renewable Power': [GHG_Power_TWh], 'Industry': [Industry_Power], 'Waste': [Waste_Power]})

    # Convert dataframes to a format that can be sent as a JSON response
    GHG_df_result = GHG_df.to_dict(orient='records')
    Power_df_result = Power_df.to_dict(orient='records')
    power_requirements_result = power_requirements.to_dict(orient='records')

    # Send response
    return jsonify({
        'GHG_df': GHG_df_result,
        'Power_df': Power_df_result,
        'power_requirements': power_requirements_result
    })
